hit X number of reports 
update scaling factor 
save pickle file of most recent scaling factors

implement # python emp_study_scaling_factors.py 

In [9]:
import pandas as pd
from scipy import stats
import random

# load in pickle file of factors
# factors = {'sobeys': {'produce': 0.8}
#         , 'walmart': {'meat': 0.6,
#                      'bakery': 0.72}}




In [10]:
stores = ['freshco', 'sobeys', 'food_basics', 'walmart']

def generate_scaling_factors(study_data):
    '''geometric mean of % diff values to be our _scaling_factor_'''

    departments = study_data.department.unique()

    factors = pd.DataFrame()

    for department in departments:
        df = study_data[study_data['department'] == department]

        for store in stores: 

            geo_mean = stats.gmean(df[f'{store} % difference'], axis=0, dtype=None)

            if department == 'meat + seafood':
                factors = factors.append({'store':store, 'department':'meat', 'geo_mean':geo_mean}, ignore_index=True)
                factors = factors.append({'store':store, 'department':'seafood', 'geo_mean':geo_mean}, ignore_index=True)
            elif department == 'home goods':
                factors = factors.append({'store':store, 'department':'household_items', 'geo_mean':geo_mean}, ignore_index=True)
            elif department == 'dairy':
                factors = factors.append({'store':store, 'department':'dairy_and_eggs', 'geo_mean':geo_mean}, ignore_index=True)
            elif department == 'dry goods':
                factors = factors.append({'store':store, 'department':'pantry', 'geo_mean':geo_mean}, ignore_index=True)
                factors = factors.append({'store':store, 'department':'snacks', 'geo_mean':geo_mean}, ignore_index=True)
                factors = factors.append({'store':store, 'department':'drinks', 'geo_mean':geo_mean}, ignore_index=True)
                factors = factors.append({'store':store, 'department':'frozen', 'geo_mean':geo_mean}, ignore_index=True)
            else:
                factors = factors.append({'store':store, 'department':department, 'geo_mean':geo_mean}, ignore_index=True)

    return factors 

In [11]:
study = pd.read_csv('../empirical study.csv')
baseline = pd.read_csv('../clean_data/zehrs/zehrs_data.csv')

factors = generate_scaling_factors(study)

factors

,department,geo_mean,store
0,produce,0.753613,freshco
1,produce,0.946817,sobeys
2,produce,0.885488,food_basics
3,produce,0.847831,walmart
4,meat,0.904069,freshco
5,seafood,0.904069,freshco
6,meat,1.066879,sobeys
7,seafood,1.066879,sobeys
8,meat,1.048851,food_basics
9,seafood,1.048851,food_basics


In [12]:
updated_factors = {'sobeys': {'produce': 0.8}
        , 'walmart': {'meat': 0.6,
                     'bakery': 0.72}}

In [14]:


for store in stores:
    globals()[f"{store}_synthetic_data"] = pd.DataFrame()

    for category in factors.department.unique():
        category_data = baseline[baseline['category'] == category]

        columns = category_data.columns

        # check if scaling factor in dict
        try: 
            update_amount = updated_factors[store][category]
            print(f'{store} {category} has updated factor')
        except:
            update_amount = 0

        geo_mean = list(factors.loc[ (factors['store']==store) & (factors['department'] == category) ].geo_mean)[0]
        noise = random.uniform(0, 0.05)
        scaling_factor = geo_mean+noise+update_amount

        category_data['price'] = category_data['price']*scaling_factor
        category_data['per_unit_price'] = category_data['per_unit_price']*scaling_factor
        category_data['sale_price'] = [None]*len(category_data)
        category_data['sale_per_unit_price'] = [None]*len(category_data)
        category_data['store'] = [store]*len(category_data)
        category_data['is_sale'] = [False]*len(category_data)

        category_data = category_data[columns]

        globals()[f"{store}_synthetic_data"] = globals()[f"{store}_synthetic_data"].append(category_data, ignore_index=True)
    
    # globals()[f"{store}_synthetic_data"].pop(globals()[f"{store}_synthetic_data"].columns[0])
    # globals()[f"{store}_synthetic_data"].to_csv(f'clean_data/{store}/synthetic_data.csv', index=False)

<ipython-input-14-cb0abce0f6eb>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  category_data['price'] = category_data['price']*scaling_factor
<ipython-input-14-cb0abce0f6eb>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  category_data['per_unit_price'] = category_data['per_unit_price']*scaling_factor
<ipython-input-14-cb0abce0f6eb>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in 

sobeys has updated factor
walmart has updated factor
walmart has updated factor


In [21]:
STEP_SIZE = 0.05 # -> 5% 

store = 'freshco'
item = 'Shallots 750 g'
change = 1 # or -1 

data = pd.read_csv(f'../clean_data/{store}/{store}_data.csv')

category = list(data.loc[(data['full_product_text']==item)].category)[0]

category


'produce'

In [22]:
if store in updated_factors.keys():
    store_dict = updated_factors[store]
    if category in store_dict.keys():
        #update 
        updated_factors[store][category] += change*STEP_SIZE
    else:
        updated_factors[store][category] = change*STEP_SIZE
else:
    updated_factors[store][category] = change*STEP_SIZE

not in


In [23]:
from pickle import dump

dump(updated_factors, open('updated_factors.pkl', 'wb'))

In [27]:
from pickle import load
file = open('updated_factors.pkl', 'rb')
updated_factors = load(file)
file.close()

updated_factors

{'sobeys': {'produce': 0.8},
 'walmart': {'meat': 0.6, 'bakery': 0.72},
 'freshco': {'produce': 0.0}}